<a href="https://colab.research.google.com/github/Devdeep-J-S/Vision-Transformers-CMS/blob/main/Task_1_Electron_photon_classification_Pytorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Name : Devdeep Shetranjiwala <br>
Email ID : devdeep0702@gmail.com 

## Task 1. Electron/photon classification
Datasets:</br>
https://cernbox.cern.ch/index.php/s/AtBT8y4MiQYFcgc (photons) </br>
https://cernbox.cern.ch/index.php/s/FbXw3V4XNyYB3oA (electrons) </br>
> Description: </br>
32x32 matrices (two channels - hit energy and time) for two classes of particles electrons and photons impinging on a calorimeter
Please use a deep learning method of your choice to achieve the highest possible
classification on this dataset.

>In this task, we will use deep learning to classify two classes of particles: electrons and photons impinging on a calorimeter. We will use two datasets, one for photons and one for electrons, which contains 32x32 matrices (two channels - hit energy and time) for each particle.</br>
We will use deep learning framework PyTorch. Our goal is to achieve the highest possible classification accuracy on this dataset with a ROC AUC score of at least 0.80.
</br>
First, we will load the data and preprocess it.<br>
Data Preprocessing : </br>
We will load the datasets for photons and electrons and preprocess them. We will convert the data into numpy arrays and normalize them by dividing each pixel value by the maximum pixel value.

In [ ]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import h5py

from torchvision.transforms import CenterCrop
from torch.utils.data import Dataset, DataLoader, random_split
from sklearn.metrics import roc_auc_score

import warnings 
warnings.filterwarnings("ignore")

In [ ]:
# Setting Seed
torch.manual_seed(42)
torch.cuda.manual_seed_all(42)
torch.cuda.manual_seed(42)
torch.backends.cudnn.benchmark = False
torch.backends.cudnn.deterministic = True

In [ ]:
# Setting device to GPU
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

In [ ]:
#Getting data
import requests
url = 'https://cernbox.cern.ch/remote.php/dav/public-files/AtBT8y4MiQYFcgc/SinglePhotonPt50_IMGCROPS_n249k_RHv1.hdf5'
r = requests.get(url, allow_redirects=True)
open('photons.hdf5', 'wb').write(r.content)
url = 'https://cernbox.cern.ch/remote.php/dav/public-files/FbXw3V4XNyYB3oA/SingleElectronPt50_IMGCROPS_n249k_RHv1.hdf5'
r = requests.get(url, allow_redirects=True)
open('electrons.hdf5', 'wb').write(r.content)

In [ ]:
# Define the neural network
class Net (nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(2, 32, kernel_size=1)
        self.bn1 = nn.BatchNorm2d(32)
        self.conv2 = nn.Conv2d(32, 32, kernel_size=3, padding=1)
        self.bn2 = nn.BatchNorm2d(32)
        self.pool1 = nn.MaxPool2d(kernel_size=2)
        self.conv3 = nn.Conv2d(32, 64, kernel_size=1)
        self.bn3 = nn.BatchNorm2d(64)
        self.conv4 = nn.Conv2d(64, 64, kernel_size=3, padding=1)
        self.bn4 = nn.BatchNorm2d(64)
        self.pool2 = nn.MaxPool2d(kernel_size=2)
        self.conv5 = nn.Conv2d(64, 128, kernel_size=1)
        self.bn5 = nn.BatchNorm2d(128)
        self.conv6 = nn.Conv2d(128, 128, kernel_size=3, padding=1)
        self.bn6 = nn.BatchNorm2d(128)
        self.pool3 = nn.MaxPool2d(kernel_size=2)
        self.fc1 = nn.Linear(128 * 4 * 4, 64)
        self.dropout = nn.Dropout(0.5)
        self.fc2 = nn.Linear(64, 1)

    def forward(self, x):
        x = x.permute(0, 3, 1, 2)
        x = self.conv1(x)
        x = self.bn1(x)
        x = torch.relu(x)
        x = self.conv2(x)
        x = self.bn2(x)
        x = torch.relu(x)
        x = torch.nn.functional.dropout(x, 0.2)
        x = self.pool1(x)
        x = self.conv3(x)
        x = self.bn3(x)
        x = torch.relu(x)
        x = self.conv4(x)
        x = self.bn4(x)
        x = torch.relu(x)
        x = torch.nn.functional.dropout(x, 0.2)
        x = self.pool2(x)
        x = self.conv5(x)
        x = self.bn5(x)
        x = torch.relu(x)
        x = self.conv6(x)
        x = self.bn6(x)
        x = torch.relu(x)
        x = torch.nn.functional.dropout(x, 0.2)
        x = self.pool3(x)
        x = torch.flatten(x, 1)
        x = self.fc1(x)
        x = torch.relu(x)
        x = self.dropout(x)
        x = self.fc2(x)
        x = torch.sigmoid(x)
        
        return x

In [ ]:
# Defining the Dataset class
class Dataset(Dataset):
    def __init__(self, electrons_data, photons_data):
        self.electrons_data = electrons_data
        self.photons_data = photons_data
        self.data_key = 'X'
        self.file_electron = h5py.File(self.electrons_data, 'r')
        self.data_electron = self.file_electron[self.data_key]
        self.file_photon = h5py.File(self.photons_data, 'r')
        self.data_photon = self.file_photon[self.data_key]
        self.data = np.concatenate((self.data_electron, self.data_photon), axis=0)
        self.labels = np.concatenate((np.ones(self.data_electron.shape[0]), np.zeros(self.data_photon.shape[0])), axis=0)
        self.labels = np.expand_dims(self.labels, axis=1)
        # wanted to try to normalize data 
        #self.data = (self.data - mean)/std


    def __len__(self):
        return self.data.shape[0]

    def __getitem__(self, idx):
        return torch.from_numpy(self.data[idx]), torch.from_numpy(self.labels[idx])

    def close(self):
        self.file_electrons.close()
        self.file_photon.close()

In [ ]:
# Making training, validation and test datasets
# train data : 80%,
# validation data : 10%,
# testing data : 10%

electrons_data = 'electrons.hdf5'
photons_data = 'photons.hdf5'
dataset = Dataset(electrons_data, photons_data)

# Split the data into training, testing sets and validating sets 
train_size = np.int32(0.8 * len(dataset))
test_size = np.int32(0.1 * len(dataset))
val_size = len(dataset) - train_size - test_size

train_data, test_data, val_data = random_split(dataset, [train_size, test_size, val_size])

# Create the data loaders
train_loader = DataLoader(train_data, batch_size=200, shuffle=True, num_workers=2)
test_loader = DataLoader(test_data, batch_size=200, shuffle=True, num_workers=2)
val_loader = DataLoader(val_data, batch_size=200, shuffle=True, num_workers=2)

In [ ]:
# Defining the loss function, optimizer 
model = Net().to(device)
criterion = nn.BCELoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.8)

In [ ]:
# Training

epochs = 200 # takes time but gives good result 
min_val_loss = np.inf

train_losses = []
val_losses = []
train_aucs = []
val_aucs = []
counter = 0

for e in range(epochs):
    train_loss = 0.0
    train_auc = 0.0
    i = 0
    for data in train_loader: # Training Loop
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        #print(inputs.shape)
        outputs = model(inputs)
        labels , outputs = labels.type(torch.FloatTensor),outputs.type(torch.FloatTensor)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()
        with torch.no_grad():
            train_auc += roc_auc_score(labels.numpy(), outputs.numpy())
        
        if i % 100 == 99:    # save every 100 mini-batches
            train_losses.append(train_loss / 100)
            train_aucs.append(train_auc / 100)
            train_loss = 0.0
            train_auc = 0.0
        
        i += 1
        
    with torch.no_grad():
        val_loss = 0.0
        val_auc = 0.0
        model.eval()
        for data in val_loader: # Validation Loop
            inputs, labels = data
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            labels , outputs = labels.type(torch.FloatTensor),outputs.type(torch.FloatTensor)
            loss = criterion(outputs,labels)
            val_loss += loss.item()
            val_auc += roc_auc_score(labels.numpy(), outputs.numpy())
        
    print(f'Epoch {e+1} Val Loss: {val_loss / len(val_loader)} \t\t Val Accuracy: {val_auc / len(val_loader)}')
    
    val_losses.append(val_loss / len(val_loader))
    val_aucs.append(val_auc / len(val_loader))
     
    # basic need for task - 1 AUC ROC SCORE    
#     if (val_auc / len(val_loader)>=0.85) :
#         break
        
    if min_val_loss > val_loss:
        print(f'Validation Loss Decreased({min_val_loss:.6f}--->{val_loss:.6f}) \t Saving The Model')
        min_val_loss = val_loss
        counter = 0 
            
        # Saving the model
        torch.save(model.state_dict(), 'saved_model.pth')
        
    else:
        # Early Stopping
        counter += 1
        if counter >= 20:
            print('Training Stopped.')
            break

Epoch 1 Val Loss: 0.6072167335265133 		 Val Accuracy: 0.7369851546367092
Validation Loss Decreased(inf--->151.196967) 	 Saving The Model
Epoch 2 Val Loss: 0.6081583854185051 		 Val Accuracy: 0.740356061553586
Epoch 3 Val Loss: 0.5914301632877335 		 Val Accuracy: 0.7503892535118144
Validation Loss Decreased(151.196967--->147.266111) 	 Saving The Model
Epoch 4 Val Loss: 0.600578661424568 		 Val Accuracy: 0.7381515964134102
Epoch 5 Val Loss: 0.5842341525727007 		 Val Accuracy: 0.7579935904893578
Validation Loss Decreased(147.266111--->145.474304) 	 Saving The Model
Epoch 6 Val Loss: 0.5934161859822561 		 Val Accuracy: 0.7548254557609606
Epoch 7 Val Loss: 0.5819701387221554 		 Val Accuracy: 0.7637875211259614
Validation Loss Decreased(145.474304--->144.910565) 	 Saving The Model
Epoch 8 Val Loss: 0.5812393273694448 		 Val Accuracy: 0.7645595698981348
Validation Loss Decreased(144.910565--->144.728593) 	 Saving The Model
Epoch 9 Val Loss: 0.5788110372531845 		 Val Accuracy: 0.76520077817863

In [ ]:
# Performance on test set
trained_model = Net().to(device)

trained_model.load_state_dict(torch.load('saved_model.pth'))
trained_model.eval()

with torch.no_grad():
    test_loss = 0.0
    test_auc = 0.0
    for data in test_loader:
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = trained_model(inputs)
        labels , outputs = labels.type(torch.FloatTensor),outputs.type(torch.FloatTensor)
        loss = criterion(outputs,labels)
        test_loss += loss.item()
        test_auc += roc_auc_score(labels.numpy(), outputs.numpy())

print(f"The loss on testing data is {test_loss/len(test_loader)} and the ROC-AUC is {test_auc/len(test_loader)}")

The loss on testing data is 0.5423456724867763 and the ROC-AUC is 0.8003556647834663


Best ROC AUC score (validate) : 0.8083 </br>
Best ROC AUC score (test) : 0.8004 </br>